# Quantitative Stock Selection

In [30]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm

## Functions

In [31]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

## Data Import and Wrangling

In [32]:
# IMPORT LATEST DATA 

data_tmp = pd.read_excel("equity_data/Borsdata_2021-05-30.xlsx", sheet_name="Export")
data_tmp = data_tmp[1:]

data_tmp = data_tmp.rename({'Performance - Perform. 3m' : 'Return 3m','Performance - Perform. 6m' : 'Return 6m',
                            'Performance - Perform. 1 y' : 'Return 1y',
                            'Div. Yield - Current': 'Yield', 
                            'Total Equity  - Millions':'Total Equity', 'FCF - Millions': 'FCF','ROE - Current':'ROE',
                            'Volatility - St.Dev. 100d':'Volatility','Market Cap - Current': 'Market Cap', 
                            'ROC - Current':'ROC', 'Tot. Assets - Millions':'Tot. Assets', 
                            'Gross profit - Millions':'Gross profit', 'Assets Turn - Current': 'Assets Turn',
                            'P/FCF - Current':'P/FCF', 'P/E - Current':'P/E', 'P/S - Current':'P/S',
                            'P/B - Current':'P/B','EV/EBIT - Current':'EV/EBIT',
                            'Info - Country' : 'Country','F-Score - Point':'F-Score',
                            'Info - List' : 'List', 'Info - Sector' : 'Sector', 'Info - Industry' : 'Industry', 'Info - Ticker' : 'Tick'}, axis=1)

# FORMAT FOR VIEWING PLEASURE
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

In [33]:
# MAKE NUMBERS FLOATS 

data_tmp['Market Cap'] = data_tmp['Market Cap'].replace(',', '') # remove , as 1000 separator

list = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Yield', 'Market Cap', 'Volatility',
        'F-Score', 'ROC', 'ROE', 'FCF', 'Total Equity', 
        'Return 3m', 'Return 6m', 'Return 1y']

for i in list: 
    data_tmp[i] = pd.to_numeric(data_tmp[i]) # make every number numeric


data_tmp['FCFROE'] = data_tmp['FCF']/data_tmp['Total Equity'] # calc FCFROE
data_tmp['GPA'] = data_tmp['Gross profit']/data_tmp['Tot. Assets'] 

In [34]:
# REMOVE TOO SMALL COMPANIES AND FINANCIALS 

data_tmp = data_tmp.loc[data_tmp['Market Cap'] > 500]
data_tmp = data_tmp.loc[(data_tmp['List'] != 'First North') & (data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]

data_full = data_tmp.copy()
### MOMENTUM

data_mom = data_tmp.copy()
data_mom = data_mom.loc[(data_tmp['Industry'] != 'Holding Companies')]

### GESTALT
data_tmp = data_tmp.loc[(data_tmp['Sector'] != 'Financials')]
data_tmp.index = range(len(data_tmp)) # make new index after removing
data = data_tmp


## MOMENTUM


In [35]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to min
for i in ['Return 3m', 'Return 6m', 'Return 1y']:
    data_mom.loc[data_mom[i].isna() ,i] = data_mom[i].min()
    
data_mom.loc[data_mom['Volatility'].isna(),'Volatility'] = data_mom['Volatility'].median()

In [36]:
# RANK ON DIFFERENT METRICS 

#f_score_cut_off = (pd.DataFrame(data.quantile(0.2, axis=0))).loc['F-Score', 0.2]



data_mom['3m Rank'] = data_mom['Return 3m'].rank(ascending = False)
data_mom['6m Rank'] = data_mom['Return 6m'].rank(ascending = False)
data_mom['1y Rank'] = data_mom['Return 1y'].rank(ascending = False)
data_mom['Volatility Rank'] = data_mom['Volatility'].rank(ascending = True)

data_mom['Momentum Rank'] = (data_mom['Return 3m'].rank(ascending = False) + data_mom['Return 6m'].rank(ascending = False) + 
                              data_mom['Return 1y'].rank(ascending = False) + data_mom['Volatility'].rank(ascending = True)).rank(ascending = True)

#data_mom['Momentum Rank'] = (data_mom['Return 3m'].rank(ascending = False) + data_mom['Return 6m'].rank(ascending = False) + 
#                              data_mom['Return 1y'].rank(ascending = False)).rank(ascending = True)


#f_score_cut_off = (pd.DataFrame(data.quantile(0.2, axis=0))).loc['F-Score', 0.2]

#data['Momentum Rank'] = (data['Return 3m'] + data['Return 6m'] + data['Return 1y']).rank(ascending = False)

## GESTALT

In [37]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'ROC', 'ROE',
           'F-Score', 'GPA', 'Assets Turn']
for i in columns: 
    data.loc[data[i].isna() ,i] = data[i].median()

# set nan to min
for i in ['Return 3m', 'Return 6m', 'Return 1y']:
    data.loc[data[i].isna() ,i] = data[i].min()
    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B']:
    data.loc[data[i] < 0 ,i] = data[i].max()
    
# set nan yield & vol to 0
data.loc[data['Yield'].isna(),'Yield'] = 0

data.loc[data['Volatility'].isna(),'Volatility'] = data['Volatility'].median()

In [38]:
# Manage negativ equity

#data['Total Equity']
#data.loc[(data['Total Equity'] < 0), ['ROE', 'FCFROE']] = np.nan
#data.loc[(data['Total Equity'] < 0), ['ROE', 'FCFROE']]

## Rankings and Sorting

In [39]:
# RANK ON DIFFERENT METRICS 

#f_score_cut_off = (pd.DataFrame(data.quantile(0.2, axis=0))).loc['F-Score', 0.2]

data['Momentum'] = (data['Return 3m'] + data['Return 6m'] + data['Return 1y'])/3

# Ranking where lower value is better
for i in ['P/E', 'P/B', 'P/S', 'P/FCF', 'EV/EBIT']:
    data[i +' Rank'] = data[i].rank()
    
# Ranking where higher value is better
for i in ['Yield', 'ROE', 'ROC', 'FCFROE', 'Momentum', 'GPA', 'Assets Turn']:
    data[i + ' Rank'] = (-data[i]).rank()
    
# Composite ranks
data['Quality Rank'] = (data['ROE Rank'] + data['ROC Rank'] + data['FCFROE Rank'] + 
                        data['GPA Rank'] + data['Assets Turn Rank']).rank()
data['Value Rank'] = (data['P/E Rank'] + data['P/B Rank'] + data['P/S Rank'] + data['P/FCF Rank'] + 
                      data['EV/EBIT Rank'] + data['Yield Rank']).rank()
 
data['Magic Formula'] = (data['ROC Rank'] + data['EV/EBIT Rank']).rank()
data['Siegfrieds'] = (data['ROC Rank'] + data['P/B Rank']).rank()
data['Triple Sort'] = ((data['Value Rank'] + data['Momentum Rank'] + data['Quality Rank'])).rank()
data['Double Sort'] = (data['Value Rank'] + data['Quality Rank']).rank()

In [40]:
# SORTING 

# number of stocks to choose before momentum screen
trend_screen = 40
double_sort_break_off = 40

value_tmp = data.sort_values(by=['Value Rank'])
quality_tmp = data.sort_values(by=['Quality Rank'])
#### MOMENTUM DATA FRAME
momentum_tmp = data_mom.sort_values(by=['Momentum Rank'])


trending_value_tmp = value_tmp[0:trend_screen]
trending_value_tmp = trending_value_tmp.sort_values(by='Momentum Rank')
trending_quality_tmp = quality_tmp[0:trend_screen]
trending_quality_tmp = trending_quality_tmp.sort_values(by='Momentum Rank')
magic_formula_tmp = data.sort_values(by=['Magic Formula'])
siegfrieds_tmp = data.sort_values(by=['Siegfrieds'])
triple_sort_tmp = data.sort_values(by=['Triple Sort'])
double_sort = data.sort_values(by=['Double Sort'])
double_sort_mom_tmp = double_sort[0:double_sort_break_off]
double_sort_mom = double_sort_mom_tmp.sort_values(by=['Momentum Rank'])

In [41]:
# FACTOR STRATEGIES 

compact = ['Company', 'List', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'Tick']

compact_mom = ['Company','Tick', 'List', 'Momentum Rank', 'F-Score' ]

# numbers of stocks to select for diff strategies
stocks = 20
magic_formula_stocks = 10
siegfried_stocks = 10
triple_sort_stocks = 20
double_sort_mom_stocks = 10

## VALUE ## 
value = value_tmp[0:stocks]
Value = value[compact]

## QUALITY ## 
quality = quality_tmp[0:stocks]
Quality = quality[compact]

## MOMENTUM ##
momentum = momentum_tmp[0:stocks]
Momentum = momentum[compact_mom]

## TRENDING VALUE ##
trending_value = trending_value_tmp[0:stocks]
Trending_Value = trending_value[compact]

## TRENDING QUALITY ##
trending_quality = trending_quality_tmp[0:stocks]
Trending_Quality = trending_quality[compact]

## MAGIC FORMULA ##
magic_formula = magic_formula_tmp[0:magic_formula_stocks]
Magic_Formula = magic_formula[compact]

## SIEGFRIEDS ##
siegfrieds = siegfrieds_tmp[0:siegfried_stocks]
Siegfrieds = siegfrieds[compact]

## TRIPLE SORT ##
triple_sort = triple_sort_tmp[0:triple_sort_stocks]
Triple_Sort = triple_sort[compact]

## DOUBLE SORT + Momentum ##
double_sort_mom = double_sort_mom[0:double_sort_mom_stocks]
Double_Sort_Mom = double_sort_mom[compact]

In [14]:
Triple_Sort 

,Company,List,Quality Rank,Value Rank,Momentum Rank,Tick
31,Bergs Timber,Small Cap,25.00,1.00,14.00,BRG B
36,Bilia,Mid Cap,17.00,8.50,27.00,BILI A
51,Byggmax,Mid Cap,9.50,2.00,47.00,BMAX
94,Ferronordic,Mid Cap,12.00,16.00,39.00,FNM
165,Nilörngruppen,Small Cap,11.00,41.00,16.00,NIL B
134,Kindred,Large Cap,2.00,51.00,15.00,KIND SDB
75,Elanders,Mid Cap,72.00,12.00,5.00,ELAN B
68,Dedicare,Small Cap,15.00,37.00,50.00,DEDI
163,New Wave,Mid Cap,69.50,34.00,3.00,NEWA B
42,Björn Borg,Small Cap,53.00,22.00,35.00,BORG


In [15]:
Momentum

,Company,Tick,List,Momentum Rank,F-Score
264,Elos,ELOS B,Small Cap,1.00,7.00
355,Getinge,GETI B,Large Cap,2.00,9.00
120,Bilia,BILI A,Mid Cap,3.00,8.00
592,New Wave,NEWA B,Mid Cap,4.50,8.00
465,K2A,K2A B,Mid Cap,4.50,nan
736,Sagax A,SAGA A,Large Cap,6.00,nan
107,BE Group,BEGR,Small Cap,7.00,8.00
813,Stendörren,STEF B,Mid Cap,8.00,nan
882,Troax Group,TROAX,Mid Cap,9.00,6.00
367,Green Landscaping,GREEN,Small Cap,10.00,7.00


In [18]:
company('Swedish')

,Börsdata ID,Company,EV/EBIT,Tick,Industry,List,Country,Return 1y,Return 6m,Return 3m,Total Equity,FCF,ROE,F-Score,Volatility,Market Cap,ROC,Tot. Assets,Gross profit,Assets Turn,P/FCF,Info - Ticker.1,Yield,P/E,P/S,P/B,Sector,FCFROE,GPA,Momentum,P/E Rank,P/B Rank,P/S Rank,P/FCF Rank,EV/EBIT Rank,Yield Rank,ROE Rank,ROC Rank,FCFROE Rank,Momentum Rank,GPA Rank,Assets Turn Rank,Quality Rank,Value Rank,Magic Formula,Siegfrieds,Triple Sort,Double Sort
221,218,Swedish Match,17.39,SWMA,Tobacco,Large Cap,Sweden,0.19,0.11,0.27,"-5,595.00","4,698.00",-0.98,7.00,23.07,"121,762.40",0.97,"16,629.00","11,271.00",1.03,25.92,SWMA,0.02,22.37,7.11,38.47,Food and Beverage,-0.84,0.68,0.19,71.00,244.00,203.00,115.00,66.00,68.00,241.00,3.00,238.00,151.00,19.00,71.00,122.00,130.00,20.00,138.50,155.00,139.00


## Market Cap test

In [15]:
# IMPORT LATEST DATA 

#data_tmp = pd.read_excel("../R-projects/Investment/Stocks/Data/Borsdata_2020-06-01-2.xlsx")
data_tmp = pd.read_excel("equity_data/Borsdata_2021-02-25.xlsx")
data_tmp = data_tmp[1:]

data_tmp = data_tmp.rename({'Performance' : 'Return 3m','Performance.1' : 'Return 6m',
                            'Performance.2' : 'Return 1y', 'Div. Yield': 'Yield', 'Info' : 'Country',
                            'Info.1' : 'List', 'Info.2' : 'Sector', 'Info.3' : 'Industry', 'Info.4' : 'Tick'}, axis=1)

# FORMAT FOR VIEWING PLEASURE
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

data_tmp['Market Cap'] = data_tmp['Market Cap'].replace(',', '') # remove , as 1000 separator


list = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Yield', 'Market Cap', 'Volatility',
        'F-Score', 'ROC', 'ROE', 'FCF', 'Total Equity ', 
        'Return 3m', 'Return 6m', 'Return 1y']

for i in list: 
    data_tmp[i] = pd.to_numeric(data_tmp[i]) # make every number numeric


sthlm_tmp = data_tmp.loc[(data_tmp['List'] != 'First North') & (data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]
full_tmp = data_tmp.loc[(data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]


In [16]:
(pd.DataFrame(sthlm_tmp.quantile(0.2, axis=0))).loc['Market Cap', 0.2]

1174.80069

In [17]:
(pd.DataFrame(full_tmp.quantile(0.5, axis=0))).loc['Market Cap', 0.5]

1178.74175

In [18]:
len(full_tmp)

675

In [19]:
len(sthlm_tmp)

341